In [1]:
import io
import os
import glob
import imageio
import pandas as pd
import numpy as np
import pathlib as pl
from tqdm import tqdm
from moviepy.editor import *
import matplotlib.pyplot as plt
from IPython.display import Image
from collections import defaultdict

In [2]:
current_path    = pl.Path.cwd() # current working directory
select_folder   = 2 # 1: train, 2: test
folder_path     = current_path.joinpath('train' if select_folder == 1 else 'test')

images_path     = folder_path.joinpath('ims')
mask_path       = folder_path.joinpath('masks')
images_list     = list(images_path.glob('*.npy'))
mask_list       = list(mask_path.glob('*.npy'))

def sorting_key(filepath):
    parts           = filepath.stem.split('_')  # Divide el nombre del archivo en partes
    patient_number  = int(parts[1])  # Extrae el número del paciente
    slice_number    = int(parts[2])  # Extrae el número del slice
    return (patient_number, slice_number)  # Retorna una tupla con ambos números

# Ordena images_list basado en el número del paciente y el número del slice
images_list.sort(key=sorting_key)
mask_list.sort(key=sorting_key)

In [3]:
def generar_imagenes_png(images_list, mask_list, cmap='gray'):
    """
    Display images and masks side by side.
    
    Parameters:
    - images_list: List of paths to image files.
    - mask_list: List of paths to mask files.
    - cmap: Color map for displaying images and masks. Default is 'gray'.
    
    Returns:
    - None
    """

    for image, mask in tqdm(zip(images_list, mask_list), desc='Displaying images and masks'):
        parts = image.stem.split('_')
        set_folder = parts[0]
        pacient = int(parts[1])
        slice_number = int(parts[2])
        # Cargar datos de imagen y máscara
        image_data = np.load(image)
        mask_data = np.load(mask)
        
        # Crear subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
        
        # Mostrar imagen
        ax1.imshow(image_data, cmap=cmap)
        ax1.set_title(f'Imagen: {image.stem}')
        ax1.axis('off')
        
        # Mostrar máscara
        ax2.imshow(mask_data, cmap=cmap)
        ax2.set_title(f'Máscara: {mask.stem}')
        ax2.axis('off')
        
        # Añadir un súper título con la información del paciente
        fig.suptitle(f'Paciente: {pacient}', fontsize=16)
        
        plt.savefig(f'{image.stem}.png')
        plt.close()

In [4]:
generar_imagenes_png(images_list[:10], mask_list[:10], cmap='gray')

Displaying images and masks: 10it [00:01,  6.53it/s]


In [5]:
def display_images_and_masks(images_list, mask_list, num_slices, counter, pacient, cmap='gray'):
    """
    Display images and masks side by side and store them in a list.
    
    Parameters:
    - images_list: List of paths to image files.
    - mask_list: List of paths to mask files.
    - num_slices: Number of image-mask pairs to display.
    - cmap: Color map for displaying images and masks. Default is 'gray'.
    
    Returns:
    - images_for_gif: List of images for creating a GIF.
    """
    num_slices = num_slices + counter
    
    # Limitar el número de imágenes y máscaras a mostrar
    images_list = images_list[counter:num_slices]
    mask_list = mask_list[counter:num_slices]
    
    # Asegurarse de que hay el mismo número de imágenes y máscaras
    assert len(images_list) == len(mask_list), "La cantidad de imágenes y máscaras debe ser la misma."
    
    images_for_gif = []  # Lista para almacenar imágenes para el GIF
    
    for image, mask in tqdm(zip(images_list, mask_list), desc='Displaying images and masks'):
        # Cargar datos de imagen y máscara
        image_data = np.load(image)
        mask_data = np.load(mask)
        
        # Crear subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
        
        # Mostrar imagen
        ax1.imshow(image_data, cmap=cmap)
        ax1.set_title(f'Imagen: {image.stem}')
        ax1.axis('off')
        
        # Mostrar máscara
        ax2.imshow(mask_data, cmap=cmap)
        ax2.set_title(f'Máscara: {mask.stem}')
        ax2.axis('off')
        
        # Añadir un súper título con la información del paciente
        fig.suptitle(f'Paciente: {pacient:03d}', fontsize=16)
        
        # Guardar la figura en un buffer y luego agregarla a images_for_gif
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        images_for_gif.append(imageio.imread(buf))
        buf.close()
        
        # Cerrar la figura para evitar la visualización
        plt.close(fig)
    
    return images_for_gif

In [6]:
def get_max_slices(images_list):
    slices_per_patient = defaultdict(int)

    for image_path in images_list:
        parts = image_path.stem.split('_')
        patient_number = int(parts[1])
        slice_number = int(parts[2])+1

        # Actualizar el conteo de slices si el número de slice actual es mayor que el máximo registrado
        slices_per_patient[patient_number] = max(slices_per_patient[patient_number], slice_number)

    return slices_per_patient

def print_max_slices(max_slices_per_patient):
    for patient_number, slice_count in max_slices_per_patient.items():
        print(f"Paciente {str(patient_number).zfill(3)}: {slice_count} slices")

# Llamar a la función y obtener el número máximo de slices por paciente
max_slices_per_patient = get_max_slices(images_list)

# Llamar a la función print_max_slices para imprimir la salida formateada
print_max_slices(max_slices_per_patient)

Paciente 050: 60 slices
Paciente 051: 76 slices
Paciente 052: 27 slices
Paciente 053: 36 slices
Paciente 054: 68 slices
Paciente 055: 59 slices
Paciente 056: 49 slices
Paciente 057: 25 slices
Paciente 058: 61 slices
Paciente 059: 52 slices


In [7]:
def create_dataframe(max_slices_per_patient):
    # Convertir el diccionario a una lista de tuples
    data_list = [(str(patient_number).zfill(3), slice_count) for patient_number, slice_count in max_slices_per_patient.items()]
    
    # Crear un DataFrame y retornarlo
    return pd.DataFrame(data_list, columns=['Paciente', 'max_slices'])

df = create_dataframe(max_slices_per_patient)

df['max_slices'].describe()

count    10.000000
mean     51.300000
std      17.127301
min      25.000000
25%      39.250000
50%      55.500000
75%      60.750000
max      76.000000
Name: max_slices, dtype: float64

In [8]:
df

,Paciente,max_slices
0,050,60
1,051,76
2,052,27
3,053,36
4,054,68
5,055,59
6,056,49
7,057,25
8,058,61
9,059,52


In [9]:
counter = 0
for patient_number, max_slices in tqdm(max_slices_per_patient.items(), desc='Processing patients'):
    #print(f'Paciente {patient_number:03d}: {max_slices} slices')
    images_for_gif = display_images_and_masks(images_list, mask_list, max_slices, counter,patient_number,cmap='gray')
    counter = counter + max_slices + 1
    if images_for_gif:
        imageio.mimsave(f'paciente_{patient_number:03d}.gif', images_for_gif, duration=0.5)
    else:
        print(f'No se generaron imágenes para el paciente {patient_number:03d}')


Displaying images and masks: 60it [00:08,  6.91it/s] ?it/s]
Displaying images and masks: 76it [00:10,  7.22it/s]:23,  9.30s/it]
Displaying images and masks: 27it [00:03,  6.82it/s]:23, 10.45s/it]
Displaying images and masks: 36it [00:05,  6.67it/s]:53,  7.60s/it]
Displaying images and masks: 68it [00:09,  7.10it/s]:41,  6.88s/it]
Displaying images and masks: 59it [00:08,  7.04it/s]:40,  8.09s/it]
Displaying images and masks: 49it [00:06,  7.59it/s]:33,  8.39s/it]
Displaying images and masks: 25it [00:03,  7.55it/s]:23,  7.91s/it]
Displaying images and masks: 61it [00:08,  6.84it/s]:13,  6.52s/it]
Displaying images and masks: 43it [00:05,  7.53it/s]:07,  7.46s/it]
Processing patients: 100%|██████████| 10/10 [01:15<00:00,  7.59s/it]


In [ ]:
gif_list = list(current_path.glob('*.gif'))
for gif in gif_list:
    display(Image(str(gif)))

In [ ]:
def convert_gif_to_mp4(gif_path, mp4_path):
    clip = VideoFileClip(str(gif_path))
    clip.write_videofile(str(mp4_path), codec='libx264')

# Obtener la ruta del directorio actual
current_path = pl.Path.cwd()

# Lista de archivos GIF en el directorio actual
gif_list = list(current_path.glob('*.gif'))

# Ruta de directorio donde se guardarán los archivos MP4
mp4_directory = current_path / "mp4"

# Crear el directorio mp4 si no existe
mp4_directory.mkdir(exist_ok=True)

# Convertir cada archivo GIF a MP4
for gif_path in gif_list:
    mp4_filename = f"{gif_path.stem}.mp4"
    mp4_path = mp4_directory / mp4_filename
    convert_gif_to_mp4(gif_path, mp4_path)